In [5]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error
import joblib  # Untuk menyimpan LabelEncoders

medicine = pd.read_csv("medicine.csv")

# feature engineering
medicine["record_timestamp"] = pd.to_datetime(medicine["record_timestamp"])
medicine["stockout_timestamp"] = pd.to_datetime(medicine["stockout_timestamp"])

medicine["stockout_days"] = (medicine["stockout_timestamp"] - medicine["record_timestamp"]).dt.days

medicine = medicine[medicine["stockout_days"] > 0].dropna(subset=["stockout_days"])

categorical_features = ["user_category", "zone", "medicine_name", "category", "corr_disease_1",  "corr_disease_2",  "corr_disease_3"]
for col in categorical_features:
    medicine[col] = LabelEncoder().fit_transform(medicine[col])

medicine["stock_per_visitor"] = medicine["stock"] / (medicine["avg_visitor_weekly"] + 1)  # Hindari divisi nol

features = [
    "user_category", "zone", "medicine_name", "category", "stock",
    "restock_frequency", "supplier_reliability", "disease_score_1",
    "disease_score_2", "disease_score_3", "corr_disease_1", "corr_disease_2",
    "corr_disease_3", "avg_price", "avg_visitor_weekly", "price", "stock_per_visitor"
]

X = medicine[features]
y = medicine["stockout_days"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = xgb.XGBRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=8,
    subsample=0.8,  
    colsample_bytree=0.8, 
    random_state=42
)

# Training model
model.fit(X_train, y_train)

# Prediksi
y_pred = model.predict(X_test)

# Evaluasi model
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

model.save_model("xgb_model.json")
print("✅ Model berhasil disimpan sebagai 'xgb_model.json'")

print(f"MAE: {mae:.2f} days")

✅ Model berhasil disimpan sebagai 'xgb_model.json'
MAE: 1.90 days
